In [1]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from indicnlp.tokenize import indic_tokenize
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Reshape, Flatten
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Debug function
def debug_print(message, variable=None):
    print(f"[DEBUG] {message}: {variable if variable is not None else ''}")
    
# Load the datasets
data1 = pd.read_csv(r"AWM_train.csv")
data2 = pd.read_csv(r"AWM_dev.csv")
malayalam_dataset = pd.concat([data1, data2], axis=0)

# Load the test file with labels
test_data = pd.read_csv(r"AWM_test_with_labels.csv")
debug_print(f"Test data shape", test_data.shape)
debug_print(f"Test data columns", test_data.columns)

# Define stopwords for Malayalam
malayalam_stopwords = {
    "അവന്", "അവള്", "അവര്", "അത്", "ഇത്", "ഞാന്", "നാം", "നിങ്ങള്", "അങ്ങെ",
    "ഇങ്ങെ", "എനിക്ക്", "അതിനായി", "അവിടെ", "ഇവിടെ", "നമ്മുടെ", "ഒരുപാട്",
    "ഈ", "അദ്ദേഹം", "അവള്", "ചെറിയ", "ഇനി", "പിന്നീട്", "പകരം", "ഇത്",
    "നല്ല", "ചില", "എന്താണ്", "പോലുള്ള", "തെറ്റായ", "പാടില്ല", "കാണുന്നു", "സമയം"
}

# Custom stemming function for Malayalam
def custom_malayalam_stemmer(word):
    common_suffixes = ['ആന്', 'ആള്', 'ക്ക്', 'പടി', 'ല്', 'ന്', 'ം', 'വയ്', 'ഉം', 'ഇല്', 'ല',
                       'മാന', 'കാര്', 'പട്ട', 'വെള്ളി', 'ശ്ശി', 'അവര്']
    for suffix in common_suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

# Text preprocessing (tokenization and stemming)
def preprocess_content(content):
    tokens = indic_tokenize.trivial_tokenize(content, lang='ml')
    return ' '.join([custom_malayalam_stemmer(word) for word in tokens if word not in malayalam_stopwords])

# Synonym substitution
synonym_dictionary = {
    "ഞാന്": ["എനിക്ക്", "ഞാനാണ്"],
    "നിങ്ങള്": ["നമുക്ക്", "നിനക്ക്"],
    "അവന്": ["അദ്ദേഹം", "അവന്‍റെ"]
}

def replace_synonyms(content):
    words = content.split()
    for i, word in enumerate(words):
        if word in synonym_dictionary:
            words[i] = random.choice(synonym_dictionary[word])
    return ' '.join(words)

# Random sentence rearrangement
def shuffle_sentences(text):
    sentences = text.split(" ")
    random.shuffle(sentences)
    return " ".join(sentences)

# Character-level augmentation for introducing minor changes
def modify_characters(text):
    words = text.split()
    for i, word in enumerate(words):
        if random.random() < 0.3:  # 30% chance to modify
            word = list(word)
            if random.random() < 0.5 and len(word) > 1:  # Random deletion
                del word[random.randint(0, len(word) - 1)]
            else:  # Random duplication
                idx = random.randint(0, len(word) - 1)
                word.insert(idx, word[idx])
            words[i] = ''.join(word)
    return ' '.join(words)

# Simulated back-translation (paraphrasing)
def simulate_back_translation(content):
    paraphrased_sentences = {
        "ഞാന് ഇന്ന് വന്നിരിക്കുന്നു": "ഇന്ന് ഞാന് എത്തിച്ചേർന്ന്",
        "അവന് ഒരു നല്ല സുഹൃത്ത്": "അവന് പുത്തന്‍ നല്ല സഖി"
    }
    return paraphrased_sentences.get(content, content)

# Apply multiple augmentation techniques
def apply_augmentations(text, num_augmentations=3):
    augmented_versions = []
    for _ in range(num_augmentations):
        augmented_text = text
        if random.random() < 0.5:
            augmented_text = replace_synonyms(augmented_text)
        if random.random() < 0.5:
            augmented_text = shuffle_sentences(augmented_text)
        if random.random() < 0.5:
            augmented_text = modify_characters(augmented_text)
        if random.random() < 0.5:
            augmented_text = simulate_back_translation(augmented_text)
        augmented_versions.append(augmented_text)
    return augmented_versions

# Augment the dataset
def preprocess_and_augment_data(df, num_augmentations=3):
    augmented_data = []
    for _, row in df.iterrows():
        processed_text = preprocess_content(row['Text'])
        augmented_data.append({'Text': processed_text, 'Class': row['Class']})
        for augmented_text in apply_augmentations(processed_text, num_augmentations):
            augmented_data.append({'Text': augmented_text, 'Class': row['Class']})
    return pd.DataFrame(augmented_data)

def extract_hybrid_features(df):
    debug_print("Extracting enhanced hybrid features")
    
    # Expanded emotion lexicons
    positive_words = {
        "സന്തോഷം", "സ്നേഹം", "നല്ല", "മികച്ച", "ആനന്ദം", "സുഖം", "ശാന്തി", 
        "വിജയം", "പുരോഗതി", "സമാധാനം", "ഉത്സാഹം", "പ്രതീക്ഷ", "വിശ്വാസം"
    }
    negative_words = {
        "വേദന", "ദുഖം", "ദുരിതം", "നരകം", "ചീത്ത", "സങ്കടം", "നിരാശ", 
        "പരാജയം", "ക്രോധം", "ഭയം", "അപകടം", "നഷ്ടം", "പ്രശ്നം"
    }
    
    # Intensity modifiers
    intensifiers = {
        "വളരെ", "ഏറ്റവും", "അതീവ", "അത്യന്തം", "തീരെ", "മിക്കവാറും", 
        "തികച്ചും", "പൂർണ്ണമായും", "അസാധാരണം"
    }
    
    # Negation words
    negators = {
        "അല്ല", "ഇല്ല", "വേണ്ട", "പാടില്ല", "കൂടാ", "വേണ്ടാ", 
        "ഒട്ടും", "ഒരിക്കലും", "ഒന്നും"
    }
    
    # Basic sentiment and lexical features
    df['Positive_Score'] = df['Text'].apply(lambda x: sum(word in positive_words for word in x.split()))
    df['Negative_Score'] = df['Text'].apply(lambda x: sum(word in negative_words for word in x.split()))
    df['Intensifier_Count'] = df['Text'].apply(lambda x: sum(word in intensifiers for word in x.split()))
    df['Negation_Count'] = df['Text'].apply(lambda x: sum(word in negators for word in x.split()))
    
    # Sentiment ratio features
    df['Sentiment_Ratio'] = df.apply(lambda row: 
        row['Positive_Score'] / (row['Negative_Score'] + 1), axis=1)
    
    # Statistical features
    df['Word_Count'] = df['Text'].apply(lambda x: len(x.split()))
    df['Char_Count'] = df['Text'].apply(len)
    df['Avg_Word_Length'] = df['Text'].apply(lambda x: 
        np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['Word_Length_Std'] = df['Text'].apply(lambda x: 
        np.std([len(word) for word in x.split()]) if len(x.split()) > 1 else 0)
    
    # Structural features
    df['Sentence_Count'] = df['Text'].apply(lambda x: len([s for s in x.split('.') if s.strip()]))
    df['Avg_Sentence_Length'] = df.apply(lambda row: 
        row['Word_Count'] / row['Sentence_Count'] if row['Sentence_Count'] > 0 else 0, axis=1)
    
    # Pattern-based features
    df['Question_Count'] = df['Text'].apply(lambda x: x.count('?'))
    df['Exclamation_Count'] = df['Text'].apply(lambda x: x.count('!'))

    return df

def vectorize_hybrid_features(df, vectorizer=None, scaler=None, label_encoder=None, is_training=True):
    # Custom analyzer for Malayalam text
    def custom_analyzer(text):
        words = text.split()
        unigrams = words
        bigrams = [' '.join(pair) for pair in zip(words[:-1], words[1:])]
        trigrams = [' '.join(triple) for triple in zip(words[:-2], words[1:-1], words[2:])]
        return unigrams + bigrams + trigrams
    
    if is_training:
        # Enhanced TF-IDF vectorization
        vectorizer = TfidfVectorizer(
            max_features=10000,
            analyzer=custom_analyzer,
            min_df=2,  # Minimum document frequency
            max_df=0.95,  # Maximum document frequency
            sublinear_tf=True,  # Apply sublinear TF scaling
            use_idf=True,
            smooth_idf=True,
            norm='l2'
        )
        
        # Get TF-IDF features
        tfidf_features = vectorizer.fit_transform(df['Text'])
    else:
        # Use previously fitted vectorizer
        tfidf_features = vectorizer.transform(df['Text'])
    
    # Combine with other features
    numerical_features = df[[
        'Positive_Score', 'Negative_Score', 'Intensifier_Count', 'Negation_Count',
        'Sentiment_Ratio', 'Word_Count', 'Char_Count', 'Avg_Word_Length',
        'Word_Length_Std', 'Sentence_Count', 'Avg_Sentence_Length',
        'Question_Count', 'Exclamation_Count'
    ]].values
    
    if is_training:
        # Scale numerical features
        scaler = StandardScaler()
        scaled_numerical = scaler.fit_transform(numerical_features)
    else:
        # Use previously fitted scaler
        scaled_numerical = scaler.transform(numerical_features)
    
    # Combine TF-IDF and numerical features
    hybrid_features = np.hstack((tfidf_features.toarray(), scaled_numerical))
    
    if is_training:
        # Encode labels
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(df['Class'])
    else:
        # Use previously fitted label encoder
        y = label_encoder.transform(df['Class'])
    
    if is_training:
        return hybrid_features, y, label_encoder, vectorizer, scaler
    else:
        return hybrid_features, y

# Train traditional ML models
def train_multiple_models(X_train, y_train, X_valid, y_valid):
    debug_print("Training ML models")
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
    }
    trained_models = {}
    for name, model in models.items():
        debug_print(f"Training {name}")
        model.fit(X_train, y_train)
        
        # Calculate and print validation accuracy
        y_valid_pred = model.predict(X_valid)
        valid_accuracy = accuracy_score(y_valid, y_valid_pred)
        debug_print(f"{name} validation accuracy", valid_accuracy * 100)
        
        trained_models[name] = model
        debug_print(f"{name} training completed")
    return trained_models

# Evaluate ML models
def evaluate_models(trained_models, X_test, y_test, classes=None):
    debug_print("Evaluating ML models")
    results = {}
    all_reports = {}
    
    for name, model in trained_models.items():
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = accuracy * 100
        
        # Generate classification report
        report = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
        all_reports[name] = report
        
        debug_print(f"{name} test accuracy", accuracy * 100)
        
        # Print classification report
        print(f"\nClassification Report for {name}:")
        print(classification_report(y_test, y_pred, target_names=classes))
        
        # Plot confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
        plt.title(f'Confusion Matrix for {name}')
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.savefig(f'{name}_confusion_matrix.png')
        plt.close()
        
    return results, all_reports

# Create deep learning models
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Reshape((-1, 1)),  # Reshape to (features, 1)
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Reshape((-1, 1)),  # Reshape to (features, 1)
        LSTM(64, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_dl_models(X_train, y_train, X_valid, y_valid):
    debug_print("Training deep learning models")
    
    # Get input shape from the data
    input_shape = (X_train.shape[1],)  # Changed to match feature dimensions
    num_classes = len(np.unique(y_train))
    
    dl_models = {
        'CNN': create_cnn_model(input_shape, num_classes),
        'LSTM': create_lstm_model(input_shape, num_classes),
    }
    
    trained_dl_models = {}
    for name, model in dl_models.items():
        debug_print(f"Training {name}")
        history = model.fit(
            X_train, y_train,  # No reshaping needed now
            epochs=10,
            batch_size=32,
            validation_data=(X_valid, y_valid),
            verbose=1
        )
        
        # Print validation accuracy for each epoch
        print(f"\n{name} Validation Accuracy per Epoch:")
        for epoch, acc in enumerate(history.history['val_accuracy']):
            print(f"Epoch {epoch+1}: Validation Accuracy = {acc:.4f}")
        
        # Print final validation accuracy
        final_val_acc = history.history['val_accuracy'][-1]
        debug_print(f"{name} final validation accuracy", final_val_acc * 100)
        
        trained_dl_models[name] = model
        debug_print(f"{name} training completed")
        
        # Plot training history
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title(f'{name} Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'{name} Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{name}_training_history.png')
        plt.close()
    
    return trained_dl_models

def evaluate_dl_models(trained_dl_models, X_test, y_test, classes=None):
    debug_print("Evaluating deep learning models")
    results = {}
    all_reports = {}
    
    for name, model in trained_dl_models.items():
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        results[name] = accuracy * 100
        
        # Get predictions
        y_pred_probs = model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)
        
        # Generate classification report
        report = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
        all_reports[name] = report
        
        debug_print(f"{name} test accuracy", accuracy * 100)
        
        # Print classification report
        print(f"\nClassification Report for {name}:")
        print(classification_report(y_test, y_pred, target_names=classes))
        
        # Plot confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
        plt.title(f'Confusion Matrix for {name}')
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.savefig(f'{name}_dl_confusion_matrix.png')
        plt.close()
        
    return results, all_reports

# Main execution
def main():
    debug_print("Starting main execution")
    
    # Data preprocessing and augmentation for training data
    augmented_df = preprocess_and_augment_data(malayalam_dataset, num_augmentations=3)
    debug_print(f"Columns after preprocessing: {list(augmented_df.columns)}")
    
    # Feature extraction for training data
    augmented_df = extract_hybrid_features(augmented_df)
    debug_print(f"Columns after hybrid feature extraction: {list(augmented_df.columns)}")
    
    # Vectorization for training data
    X, y, label_encoder, vectorizer, scaler = vectorize_hybrid_features(augmented_df, is_training=True)
    debug_print(f"Number of classes in training data: {len(np.unique(y))}")
    debug_print(f"Class labels: {label_encoder.classes_}")
    
    # Split the training data
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocess test data
    test_df = test_data.copy()
    debug_print(f"Test data shape before preprocessing: {test_df.shape}")
    
    # Check if test data has the required columns
    if 'Text' not in test_df.columns or 'Class' not in test_df.columns:
        debug_print("WARNING: Test data does not have the expected column names. Renaming columns.")
        if len(test_df.columns) >= 2:
            test_df.columns = ['Text', 'Class'] + list(test_df.columns[2:])
    
    # Extract features from test data
    test_df = extract_hybrid_features(test_df)
    debug_print(f"Test data columns after feature extraction: {list(test_df.columns)}")
    
    # Vectorize test data using the same vectorizer, scaler, and label encoder
    X_test, y_test = vectorize_hybrid_features(test_df, vectorizer, scaler, label_encoder, is_training=False)
    debug_print(f"Test data shape after vectorization: {X_test.shape}")
    
    # Get class names for reporting
    class_names = label_encoder.classes_
    
    # Train and evaluate ML models - now passing validation set
    trained_ml_models = train_multiple_models(X_train, y_train, X_valid, y_valid)
    ml_results, ml_reports = evaluate_models(trained_ml_models, X_test, y_test, classes=class_names)
    
    # Train and evaluate DL models
    trained_dl_models = train_dl_models(X_train, y_train, X_valid, y_valid)
    dl_results, dl_reports = evaluate_dl_models(trained_dl_models, X_test, y_test, classes=class_names)

    # Create a validation results summary table
    print("\n=== Validation Accuracy Summary ===")
    # For ML models, evaluate on validation set
    for name, model in trained_ml_models.items():
        val_pred = model.predict(X_valid)
        val_acc = accuracy_score(y_valid, val_pred) * 100
        print(f"{name}: {val_acc:.2f}%")
    
    # For DL models, get final validation accuracy
    for name, model in trained_dl_models.items():
        val_loss, val_acc = model.evaluate(X_valid, y_valid, verbose=0)
        print(f"{name}: {val_acc * 100:.2f}%")
    
    # Print combined results
    debug_print("Final Results on Test Data:")
    debug_print("ML Models:", ml_results)
    debug_print("DL Models:", dl_results)
    
    # Find the best model
    all_results = {**ml_results, **dl_results}
    best_model = max(all_results, key=all_results.get)
    debug_print(f"Best model: {best_model} with accuracy: {all_results[best_model]:.2f}%")
    
    # Plot overall results comparison
    plt.figure(figsize=(12, 6))
    bars = plt.bar(all_results.keys(), all_results.values())
    plt.title('Model Accuracy Comparison on Test Data')
    plt.xlabel('Models')
    plt.ylabel('Accuracy (%)')
    plt.xticks(rotation=45)
    
    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                 f'{height:.2f}%', ha='center', va='bottom', rotation=0)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png')
    plt.close()
    
    debug_print("Execution completed")

if __name__ == "__main__":
    main()

[DEBUG] Test data shape: (629, 3)
[DEBUG] Test data columns: Index(['id', 'Text', 'Class'], dtype='object')
[DEBUG] Starting main execution: 
[DEBUG] Columns after preprocessing: ['Text', 'Class']: 
[DEBUG] Extracting enhanced hybrid features: 
[DEBUG] Columns after hybrid feature extraction: ['Text', 'Class', 'Positive_Score', 'Negative_Score', 'Intensifier_Count', 'Negation_Count', 'Sentiment_Ratio', 'Word_Count', 'Char_Count', 'Avg_Word_Length', 'Word_Length_Std', 'Sentence_Count', 'Avg_Sentence_Length', 'Question_Count', 'Exclamation_Count']: 
[DEBUG] Number of classes in training data: 2: 
[DEBUG] Class labels: ['Abusive' 'Non-Abusive']: 
[DEBUG] Test data shape before preprocessing: (629, 3): 
[DEBUG] Extracting enhanced hybrid features: 
[DEBUG] Test data columns after feature extraction: ['id', 'Text', 'Class', 'Positive_Score', 'Negative_Score', 'Intensifier_Count', 'Negation_Count', 'Sentiment_Ratio', 'Word_Count', 'Char_Count', 'Avg_Word_Length', 'Word_Length_Std', 'Sentence

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

[DEBUG] K-Nearest Neighbors validation accuracy: 83.08771929824562
[DEBUG] K-Nearest Neighbors training completed: 
[DEBUG] Evaluating ML models: 
[DEBUG] Random Forest test accuracy: 63.11605723370429

Classification Report for Random Forest:
              precision    recall  f1-score   support

     Abusive       0.67      0.57      0.61       323
 Non-Abusive       0.60      0.70      0.65       306

    accuracy                           0.63       629
   macro avg       0.63      0.63      0.63       629
weighted avg       0.64      0.63      0.63       629

[DEBUG] Logistic Regression test accuracy: 64.06995230524643

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

     Abusive       0.69      0.55      0.61       323
 Non-Abusive       0.61      0.73      0.66       306

    accuracy                           0.64       629
   macro avg       0.65      0.64      0.64       629
weighted avg       0.65      0.64      0.64     

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

[DEBUG] K-Nearest Neighbors test accuracy: 51.51033386327504

Classification Report for K-Nearest Neighbors:
              precision    recall  f1-score   support

     Abusive       0.53      0.49      0.51       323
 Non-Abusive       0.50      0.54      0.52       306

    accuracy                           0.52       629
   macro avg       0.52      0.52      0.52       629
weighted avg       0.52      0.52      0.51       629

[DEBUG] Training deep learning models: 
[DEBUG] Training CNN: 
Epoch 1/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 0.6231 - loss: 0.6304 - val_accuracy: 0.8218 - val_loss: 0.4168
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 18s 50ms/step - accuracy: 0.8570 - loss: 0.3239 - val_accuracy: 0.8965 - val_loss: 0.2781
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 18s 50ms/step - accuracy: 0.9362 - loss: 0.1696 - val_accuracy: 0.9291 - val_loss: 0.2107
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 18s 50ms/step - accuracy: 0.9619 - loss: 0.0995 - val_accuracy: 0.9446 

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

K-Nearest Neighbors: 83.09%
CNN: 95.79%
LSTM: 54.95%
[DEBUG] Final Results on Test Data:: 
[DEBUG] ML Models:: {'Random Forest': 63.11605723370429, 'Logistic Regression': 64.06995230524643, 'Gradient Boosting': 59.77742448330684, 'K-Nearest Neighbors': 51.51033386327504}
[DEBUG] DL Models:: {'CNN': 65.97774028778076, 'LSTM': 48.33068251609802}
[DEBUG] Best model: CNN with accuracy: 65.98%: 
[DEBUG] Execution completed: 


In [ ]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from indicnlp.tokenize import indic_tokenize
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Reshape, Flatten
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Debug function
def debug_print(message, variable=None):
    print(f"[DEBUG] {message}: {variable if variable is not None else ''}")
    
# Load the datasets
data1 = pd.read_csv(r"AWT_train.csv")
data2 = pd.read_csv(r"AWT_dev.csv")
tamil_dataset = pd.concat([data1, data2], axis=0)

# Load the test file with labels
test_data = pd.read_csv(r"AWT_test_with_labels.csv")
debug_print(f"Test data shape", test_data.shape)
debug_print(f"Test data columns", test_data.columns)

tamil_dataset['Class'] = tamil_dataset['Class'].str.lower()
test_data['Class'] = test_data['Class'].str.lower()

# Define stopwords for Tamil
tamil_stopwords = {
    "அவன்", "அவள்", "அவர்கள்", "அது", "இது", "நான்", "நாம்", "நீங்கள்", "அங்கே",
    "இங்கே", "என்னை", "அதை", "இல்லை", "அதற்காக", "அதற்கு", "பின்னர்", "இப்போது",
    "இந்த", "அந்த", "பொறுப்பு", "மற்றவை", "ஒரு", "ஏன்", "எப்போது", "எப்படி",
    "காரணம்", "மற்றும்", "சில", "இல்லாமல்", "ஆனால்", "வேண்டும்", "முடியும்"
}

# Custom stemming function for Tamil
def custom_tamil_stemmer(word):
    common_suffixes = ['கள்', 'ங்கள்', 'ஆக', 'ல்', 'ன்', 'ம்', 'ஆன்', 'ஆள்', 'க்கு', 
                      'உம்', 'இல்', 'இன்', 'ஆல்', 'உடன்', 'அது', 'இது', 'ஓடு']
    for suffix in common_suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

# Text preprocessing (tokenization and stemming)
def preprocess_content(content):
    tokens = indic_tokenize.trivial_tokenize(content, lang='ta')
    return ' '.join([custom_tamil_stemmer(word) for word in tokens if word not in tamil_stopwords])

# Synonym substitution
synonym_dictionary = {
    "நான்": ["எனக்கு", "நானாக"],
    "நீங்கள்": ["நீ", "உங்களுக்கு"],
    "அவன்": ["அவனுக்கு", "இவன்"]
}

def replace_synonyms(content):
    words = content.split()
    for i, word in enumerate(words):
        if word in synonym_dictionary:
            words[i] = random.choice(synonym_dictionary[word])
    return ' '.join(words)

# Random sentence rearrangement
def shuffle_sentences(text):
    sentences = text.split(" ")
    random.shuffle(sentences)
    return " ".join(sentences)

# Character-level augmentation for introducing minor changes
def modify_characters(text):
    words = text.split()
    for i, word in enumerate(words):
        if random.random() < 0.3:  # 30% chance to modify
            word = list(word)
            if random.random() < 0.5 and len(word) > 1:  # Random deletion
                del word[random.randint(0, len(word) - 1)]
            else:  # Random duplication
                idx = random.randint(0, len(word) - 1)
                word.insert(idx, word[idx])
            words[i] = ''.join(word)
    return ' '.join(words)

# Simulated back-translation (paraphrasing)
def simulate_back_translation(content):
    paraphrased_sentences = {
        "நான் இன்று வந்தேன்": "இன்று நான் வந்துவிட்டேன்",
        "அவன் நல்ல நண்பன்": "அவன் சிறந்த தோழன்"
    }
    return paraphrased_sentences.get(content, content)

# Apply multiple augmentation techniques
def apply_augmentations(text, num_augmentations=3):
    augmented_versions = []
    for _ in range(num_augmentations):
        augmented_text = text
        if random.random() < 0.5:
            augmented_text = replace_synonyms(augmented_text)
        if random.random() < 0.5:
            augmented_text = shuffle_sentences(augmented_text)
        if random.random() < 0.5:
            augmented_text = modify_characters(augmented_text)
        if random.random() < 0.5:
            augmented_text = simulate_back_translation(augmented_text)
        augmented_versions.append(augmented_text)
    return augmented_versions

# Augment the dataset
def preprocess_and_augment_data(df, num_augmentations=3):
    augmented_data = []
    for _, row in df.iterrows():
        processed_text = preprocess_content(row['Text'])
        augmented_data.append({'Text': processed_text, 'Class': row['Class']})
        for augmented_text in apply_augmentations(processed_text, num_augmentations):
            augmented_data.append({'Text': augmented_text, 'Class': row['Class']})
    return pd.DataFrame(augmented_data)

def extract_hybrid_features(df):
    debug_print("Extracting enhanced hybrid features")
    
    # Expanded emotion lexicons for Tamil
    positive_words = {
        "மகிழ்ச்சி", "அன்பு", "நல்ல", "சிறந்த", "இன்பம்", "சந்தோஷம்", "அமைதி",
        "வெற்றி", "முன்னேற்றம்", "நம்பிக்கை", "உற்சாகம்", "எதிர்பார்ப்பு", "நம்பிக்கை"
    }
    negative_words = {
        "வலி", "துக்கம்", "துன்பம்", "கோபம்", "மோசம்", "கவலை", "ஏமாற்றம்",
        "தோல்வி", "பயம்", "ஆபத்து", "இழப்பு", "பிரச்சனை", "வருத்தம்"
    }
    
    # Intensity modifiers for Tamil
    intensifiers = {
        "மிகவும்", "அதிகமாக", "முற்றிலும்", "மிக", "நன்றாக", "பெரும்பாலும்",
        "முழுவதும்", "அசாதாரணமாக"
    }
    
    # Negation words for Tamil
    negators = {
        "இல்லை", "வேண்டாம்", "கூடாது", "முடியாது", "மாட்டேன்",
        "ஒருபோதும்", "எதுவும்"
    }
    
    # Basic sentiment and lexical features
    df['Positive_Score'] = df['Text'].apply(lambda x: sum(word in positive_words for word in x.split()))
    df['Negative_Score'] = df['Text'].apply(lambda x: sum(word in negative_words for word in x.split()))
    df['Intensifier_Count'] = df['Text'].apply(lambda x: sum(word in intensifiers for word in x.split()))
    df['Negation_Count'] = df['Text'].apply(lambda x: sum(word in negators for word in x.split()))
    
    # Sentiment ratio features
    df['Sentiment_Ratio'] = df.apply(lambda row: 
        row['Positive_Score'] / (row['Negative_Score'] + 1), axis=1)
    
    # Statistical features
    df['Word_Count'] = df['Text'].apply(lambda x: len(x.split()))
    df['Char_Count'] = df['Text'].apply(len)
    df['Avg_Word_Length'] = df['Text'].apply(lambda x: 
        np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['Word_Length_Std'] = df['Text'].apply(lambda x: 
        np.std([len(word) for word in x.split()]) if len(x.split()) > 1 else 0)
    
    # Structural features
    df['Sentence_Count'] = df['Text'].apply(lambda x: len([s for s in x.split('.') if s.strip()]))
    df['Avg_Sentence_Length'] = df.apply(lambda row: 
        row['Word_Count'] / row['Sentence_Count'] if row['Sentence_Count'] > 0 else 0, axis=1)
    
    # Pattern-based features
    df['Question_Count'] = df['Text'].apply(lambda x: x.count('?'))
    df['Exclamation_Count'] = df['Text'].apply(lambda x: x.count('!'))

    return df

def vectorize_hybrid_features(df, vectorizer=None, scaler=None, label_encoder=None, is_training=True):
    # Custom analyzer for Tamil text
    def custom_analyzer(text):
        words = text.split()
        unigrams = words
        bigrams = [' '.join(pair) for pair in zip(words[:-1], words[1:])]
        trigrams = [' '.join(triple) for triple in zip(words[:-2], words[1:-1], words[2:])]
        return unigrams + bigrams + trigrams
    
    if is_training:
        # Enhanced TF-IDF vectorization
        vectorizer = TfidfVectorizer(
            max_features=10000,
            analyzer=custom_analyzer,
            min_df=2,  # Minimum document frequency
            max_df=0.95,  # Maximum document frequency
            sublinear_tf=True,  # Apply sublinear TF scaling
            use_idf=True,
            smooth_idf=True,
            norm='l2'
        )
        
        # Get TF-IDF features
        tfidf_features = vectorizer.fit_transform(df['Text'])
    else:
        # Use previously fitted vectorizer
        tfidf_features = vectorizer.transform(df['Text'])
    
    # Combine with other features
    numerical_features = df[[
        'Positive_Score', 'Negative_Score', 'Intensifier_Count', 'Negation_Count',
        'Sentiment_Ratio', 'Word_Count', 'Char_Count', 'Avg_Word_Length',
        'Word_Length_Std', 'Sentence_Count', 'Avg_Sentence_Length',
        'Question_Count', 'Exclamation_Count'
    ]].values
    
    if is_training:
        # Scale numerical features
        scaler = StandardScaler()
        scaled_numerical = scaler.fit_transform(numerical_features)
    else:
        # Use previously fitted scaler
        scaled_numerical = scaler.transform(numerical_features)
    
    # Combine TF-IDF and numerical features
    hybrid_features = np.hstack((tfidf_features.toarray(), scaled_numerical))
    
    if is_training:
        # Encode labels
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(df['Class'])
    else:
        # Use previously fitted label encoder
        y = label_encoder.transform(df['Class'])
    
    if is_training:
        return hybrid_features, y, label_encoder, vectorizer, scaler
    else:
        return hybrid_features, y

# Train traditional ML models
def train_multiple_models(X_train, y_train, X_valid, y_valid):
    debug_print("Training ML models")
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
    }
    trained_models = {}
    for name, model in models.items():
        debug_print(f"Training {name}")
        model.fit(X_train, y_train)
        
        # Calculate and print validation accuracy
        y_valid_pred = model.predict(X_valid)
        valid_accuracy = accuracy_score(y_valid, y_valid_pred)
        debug_print(f"{name} validation accuracy", valid_accuracy * 100)
        
        trained_models[name] = model
        debug_print(f"{name} training completed")
    return trained_models

# Evaluate ML models
def evaluate_models(trained_models, X_test, y_test, classes=None):
    debug_print("Evaluating ML models")
    results = {}
    all_reports = {}
    
    for name, model in trained_models.items():
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = accuracy * 100
        
        # Generate classification report
        report = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
        all_reports[name] = report
        
        debug_print(f"{name} test accuracy", accuracy * 100)
        
        # Print classification report
        print(f"\nClassification Report for {name}:")
        print(classification_report(y_test, y_pred, target_names=classes))
        
        # Plot confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
        plt.title(f'Confusion Matrix for {name}')
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.savefig(f'{name}_confusion_matrix.png')
        plt.close()
        
    return results, all_reports

# Create deep learning models
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Reshape((-1, 1)),  # Reshape to (features, 1)
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Reshape((-1, 1)),  # Reshape to (features, 1)
        LSTM(64, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_dl_models(X_train, y_train, X_valid, y_valid):
    debug_print("Training deep learning models")
    
    # Get input shape from the data
    input_shape = (X_train.shape[1],)  # Changed to match feature dimensions
    num_classes = len(np.unique(y_train))
    
    dl_models = {
        'CNN': create_cnn_model(input_shape, num_classes),
        'LSTM': create_lstm_model(input_shape, num_classes),
    }
    
    trained_dl_models = {}
    for name, model in dl_models.items():
        debug_print(f"Training {name}")
        history = model.fit(
            X_train, y_train,  # No reshaping needed now
            epochs=10,
            batch_size=32,
            validation_data=(X_valid, y_valid),
            verbose=1
        )
        
        # Print validation accuracy for each epoch
        print(f"\n{name} Validation Accuracy per Epoch:")
        for epoch, acc in enumerate(history.history['val_accuracy']):
            print(f"Epoch {epoch+1}: Validation Accuracy = {acc:.4f}")
        
        # Print final validation accuracy
        final_val_acc = history.history['val_accuracy'][-1]
        debug_print(f"{name} final validation accuracy", final_val_acc * 100)
        
        trained_dl_models[name] = model
        debug_print(f"{name} training completed")
        
        # Plot training history
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title(f'{name} Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'{name} Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{name}_training_history.png')
        plt.close()
    
    return trained_dl_models

def evaluate_dl_models(trained_dl_models, X_test, y_test, classes=None):
    debug_print("Evaluating deep learning models")
    results = {}
    all_reports = {}
    
    for name, model in trained_dl_models.items():
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        results[name] = accuracy * 100
        
        # Get predictions
        y_pred_probs = model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)
        
        # Generate classification report
        report = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
        all_reports[name] = report
        
        debug_print(f"{name} test accuracy", accuracy * 100)
        
        # Print classification report
        print(f"\nClassification Report for {name}:")
        print(classification_report(y_test, y_pred, target_names=classes))
        
        # Plot confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
        plt.title(f'Confusion Matrix for {name}')
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.savefig(f'{name}_dl_confusion_matrix.png')
        plt.close()
        
    return results, all_reports

# Main execution
def main():
    debug_print("Starting main execution")
    
    # Data preprocessing and augmentation for training data
    augmented_df = preprocess_and_augment_data(tamil_dataset, num_augmentations=3)
    debug_print(f"Columns after preprocessing: {list(augmented_df.columns)}")
    
    # Feature extraction for training data
    augmented_df = extract_hybrid_features(augmented_df)
    debug_print(f"Columns after hybrid feature extraction: {list(augmented_df.columns)}")
    
    # Vectorization for training data
    X, y, label_encoder, vectorizer, scaler = vectorize_hybrid_features(augmented_df, is_training=True)
    debug_print(f"Number of classes in training data: {len(np.unique(y))}")
    debug_print(f"Class labels: {label_encoder.classes_}")
    
    # Split the training data
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocess test data
    test_df = test_data.copy()
    debug_print(f"Test data shape before preprocessing: {test_df.shape}")
    
    # Check if test data has the required columns
    if 'Text' not in test_df.columns or 'Class' not in test_df.columns:
        debug_print("WARNING: Test data does not have the expected column names. Renaming columns.")
        if len(test_df.columns) >= 2:
            test_df.columns = ['Text', 'Class'] + list(test_df.columns[2:])
    
    # Extract features from test data
    test_df = extract_hybrid_features(test_df)
    debug_print(f"Test data columns after feature extraction: {list(test_df.columns)}")
    
    # Vectorize test data using the same vectorizer, scaler, and label encoder
    X_test, y_test = vectorize_hybrid_features(test_df, vectorizer, scaler, label_encoder, is_training=False)
    debug_print(f"Test data shape after vectorization: {X_test.shape}")
    
    # Get class names for reporting
    class_names = label_encoder.classes_
    
    # Train and evaluate ML models - now passing validation set
    trained_ml_models = train_multiple_models(X_train, y_train, X_valid, y_valid)
    ml_results, ml_reports = evaluate_models(trained_ml_models, X_test, y_test, classes=class_names)
    
    # Train and evaluate DL models
    trained_dl_models = train_dl_models(X_train, y_train, X_valid, y_valid)
    dl_results, dl_reports = evaluate_dl_models(trained_dl_models, X_test, y_test, classes=class_names)

    # Create a validation results summary table
    print("\n=== Validation Accuracy Summary ===")
    # For ML models, evaluate on validation set
    for name, model in trained_ml_models.items():
        val_pred = model.predict(X_valid)
        val_acc = accuracy_score(y_valid, val_pred) * 100
        print(f"{name}: {val_acc:.2f}%")
    
    # For DL models, get final validation accuracy
    for name, model in trained_dl_models.items():
        val_loss, val_acc = model.evaluate(X_valid, y_valid, verbose=0)
        print(f"{name}: {val_acc * 100:.2f}%")
    
    # Print combined results
    debug_print("Final Results on Test Data:")
    debug_print("ML Models:", ml_results)
    debug_print("DL Models:", dl_results)
    
    # Find the best model
    all_results = {**ml_results, **dl_results}
    best_model = max(all_results, key=all_results.get)
    debug_print(f"Best model: {best_model} with accuracy: {all_results[best_model]:.2f}%")
    
    # Plot overall results comparison
    plt.figure(figsize=(12, 6))
    bars = plt.bar(all_results.keys(), all_results.values())
    plt.title('Model Accuracy Comparison on Test Data')
    plt.xlabel('Models')
    plt.ylabel('Accuracy (%)')
    plt.xticks(rotation=45)
    
    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                 f'{height:.2f}%', ha='center', va='bottom', rotation=0)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png')
    plt.close()
    
    debug_print("Execution completed")

if __name__ == "__main__":
    main()

[DEBUG] Test data shape: (598, 3)
[DEBUG] Test data columns: Index(['id', 'Text', 'Class'], dtype='object')
[DEBUG] Starting main execution: 
[DEBUG] Columns after preprocessing: ['Text', 'Class']: 
[DEBUG] Extracting enhanced hybrid features: 
[DEBUG] Columns after hybrid feature extraction: ['Text', 'Class', 'Positive_Score', 'Negative_Score', 'Intensifier_Count', 'Negation_Count', 'Sentiment_Ratio', 'Word_Count', 'Char_Count', 'Avg_Word_Length', 'Word_Length_Std', 'Sentence_Count', 'Avg_Sentence_Length', 'Question_Count', 'Exclamation_Count']: 
[DEBUG] Number of classes in training data: 2: 
[DEBUG] Class labels: ['abusive' 'non-abusive']: 
[DEBUG] Test data shape before preprocessing: (598, 3): 
[DEBUG] Extracting enhanced hybrid features: 
[DEBUG] Test data columns after feature extraction: ['id', 'Text', 'Class', 'Positive_Score', 'Negative_Score', 'Intensifier_Count', 'Negation_Count', 'Sentiment_Ratio', 'Word_Count', 'Char_Count', 'Avg_Word_Length', 'Word_Length_Std', 'Sentence

I0000 00:00:1744480798.398713      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


[DEBUG] Training CNN: 
Epoch 1/10


I0000 00:00:1744480803.243446     115 service.cc:148] XLA service 0x792efc007dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744480803.244044     115 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1744480803.506311     115 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/339 ━━━━━━━━━━━━━━━━━━━━ 17s 53ms/step - accuracy: 0.5312 - loss: 0.6965

I0000 00:00:1744480807.213503     115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


339/339 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - accuracy: 0.6682 - loss: 0.5931 - val_accuracy: 0.8429 - val_loss: 0.3496
Epoch 2/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.8819 - loss: 0.2666 - val_accuracy: 0.9129 - val_loss: 0.2088
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.9519 - loss: 0.1232 - val_accuracy: 0.9473 - val_loss: 0.1433
Epoch 4/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 17s 52ms/step - accuracy: 0.9718 - loss: 0.0711 - val_accuracy: 0.9598 - val_loss: 0.1168
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.9832 - loss: 0.0452 - val_accuracy: 0.9627 - val_loss: 0.1268
Epoch 6/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.9848 - loss: 0.0383 - val_accuracy: 0.9664 - val_loss: 0.1047
Epoch 7/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.9907 - loss: 0.0272 - val_accuracy: 0.9694 - val_loss: 0.1140
Epoch 8/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.9899 - loss: 0.0234 - val_accurac